In [1]:
import sysconfig

from pyspark.sql import SparkSession

In [63]:
spark = (
    SparkSession
        .builder
        .remote("sc://localhost:15002")
        .appName("hive")
        .enableHiveSupport()
        .config("hive.metastore.uris", "thrift://localhost:9083")
        .config("spark.sql.warehouse.dir", "/user/hive/warehouse")
        .getOrCreate()
)

# Load the Spark Connect jar
spark_connect_jar = f"{sysconfig.get_paths()['purelib']}/pyspark/jars/spark-connect_2.13-4.0.1.jar"
spark.addArtifacts(spark_connect_jar)

/Users/mathiasgredal/git/Boston-Transport-Department/.venv/lib/python3.10/site-packages/pyspark/sql/connect/conf.py:64: UserWarning: Failed to set spark.sql.catalogImplementation to Some(hive) due to [CANNOT_MODIFY_CONFIG] Cannot modify the value of the Spark config: "spark.sql.catalogImplementation".
See also 'https://spark.apache.org/docs/latest/sql-migration-guide.html#ddl-statements'. SQLSTATE: 46110
  warnings.warn(warn)
/Users/mathiasgredal/git/Boston-Transport-Department/.venv/lib/python3.10/site-packages/pyspark/sql/connect/conf.py:64: UserWarning: Failed to set spark.sql.warehouse.dir to Some(/user/hive/warehouse) due to [CANNOT_MODIFY_CONFIG] Cannot modify the value of the Spark config: "spark.sql.warehouse.dir".
See also 'https://spark.apache.org/docs/latest/sql-migration-guide.html#ddl-statements'. SQLSTATE: 46110
  warnings.warn(warn)


In [66]:
#spark.sql("CREATE EXTERNAL TABLE IF NOT EXISTS weather_data (name VARCHAR(64), age INT, gpa DECIMAL(3, 2)) STORED AS PARQUET LOCATION 'hdfs://hdfs-cluster-namenode-default-0.hdfs-cluster-namenode-default.bigdata.svc.cluster.local/user/hive/warehouse/weater_data'")
#spark.sql("INSERT INTO students (name, age, gpa) VALUES ('Hello world123456', 20, 3.5)")
#spark.sql("REFRESH TABLE students")
spark.sql("""WITH rides AS (
  SELECT
    concat_ws('|', starttime, start_station_id, end_station_id, tripduration) AS ride_id,
    tripduration,
    starttime
  FROM bike_data
),
wx AS (
  SELECT
    observation_date,
    dry_bulb_temperature_celsius
  FROM weather_data
),
matched AS (
  SELECT
    r.ride_id,
    r.tripduration,
    r.starttime,
    w.dry_bulb_temperature_celsius,
    abs(
      unix_timestamp(r.starttime, "yyyy-MM-dd'T'HH:mm:ss") -
      unix_timestamp(w.observation_date, "yyyy-MM-dd'T'HH:mm:ss")
    ) AS diff_seconds_to_obs,
    row_number() OVER (
      PARTITION BY r.ride_id
      ORDER BY abs(
        unix_timestamp(r.starttime, "yyyy-MM-dd'T'HH:mm:ss") -
        unix_timestamp(w.observation_date, "yyyy-MM-dd'T'HH:mm:ss")
      )
    ) AS rn
  FROM rides r
  JOIN wx w
    ON w.observation_date BETWEEN
         from_unixtime(unix_timestamp(r.starttime, "yyyy-MM-dd'T'HH:mm:ss") - 3600)
     AND from_unixtime(unix_timestamp(r.starttime, "yyyy-MM-dd'T'HH:mm:ss") + 3600)
)
SELECT
  ride_id,
  tripduration,
  starttime,
  dry_bulb_temperature_celsius AS matched_temp_c,
  diff_seconds_to_obs
FROM matched
WHERE rn = 1;""")
#spark.sql("clear cache")
#spark.sql("INSERT OVERWRITE TABLE students SELECT * FROM students;")
# mydf = spark.sql("select * from weather_data")
# mydf.show()



#CREATE TABLE students ( name STRING, age INT, gpa DECIMAL(3,2) ) STORED AS PARQUET TBLPROPERTIES ('transactional'='true') LOCATION 'hdfs://hdfs-cluster-namenode-default-0.hdfs-cluster-namenode-default.bigdata.svc.cluster.local/user/hive/warehouse/students2';
#INSERT OVERWRITE TABLE students SELECT * FROM students;
#spark.sql("ALTER TABLE students3 SET TBLPROPERTIES('transactional'='true')")


DataFrame[ride_id: string, tripduration: double, starttime: string, matched_temp_c: double, diff_seconds_to_obs: bigint]